In [ ]:
# Define paths
input_path = 'path/to/input/images'
output_path = 'path/to/output/predictions.csv'
data_dir = ''

In [1]:
import argparse
from pathlib import Path
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

def load_data(csv_path, image_dir):
    df = pd.read_csv(csv_path)
    df['s'] = df['basename'].apply(lambda x: str(Path(image_dir) / x))
    return df

def create_data_generators(df, image_size=(128, 128), batch_size=32, validation_split=0.2):
    datagen = ImageDataGenerator(rescale=1./255, validation_split=validation_split)
    
    train_generator = datagen.flow_from_dataframe(
        df,
        x_col='filepath',
        y_col='deepest_name',
        target_size=image_size,
        batch_size=batch_size,
        class_mode='categorical',
        subset='training'
    )
    
    validation_generator = datagen.flow_from_dataframe(
        df,
        x_col='filepath',
        y_col='deepest_name',
        target_size=image_size,
        batch_size=batch_size,
        class_mode='categorical',
        subset='validation'
    )
    
    return train_generator, validation_generator

def build_model(input_shape, num_classes):
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(512, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])
    
    model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

def predict(model, inputs, image_size=(128, 128)):
    images = []
    for img_path in inputs:
        img = tf.keras.preprocessing.image.load_img(img_path, target_size=image_size)
        img = tf.keras.preprocessing.image.img_to_array(img)
        img = np.expand_dims(img, axis=0)
        img = img / 255.0
        images.append(img)
    
    images = np.vstack(images)
    predictions = model.predict(images)
    return predictions

def main(input_path, output_path, csv_path, image_dir):
    df = load_data(csv_path, image_dir)
    train_generator, validation_generator = create_data_generators(df)
    
    input_shape = (128, 128, 3)
    num_classes = len(train_generator.class_indices)
    
    model = build_model(input_shape, num_classes)
    
    model.fit(train_generator, validation_data=validation_generator, epochs=10)
    
    inputs = list(Path(input_path).glob("*.jpg"))
    predictions = predict(model, inputs)
    
    values = {
        "image_uid": [Path(input).stem for input in inputs],
        "predictions": [train_generator.class_indices[pred] for pred in np.argmax(predictions, axis=1)]
    }
    
    df_predictions = pd.DataFrame(values)
    df_predictions.to_csv(output_path, index=False)

if __name__ == '__main__':
    parser = argparse.ArgumentParser()
    
    parser.add_argument("input_path", help="Path to input images")
    parser.add_argument("output_path", help="Path to output csv")
    parser.add_argument("csv_path", help="Path to CSV file with labels")
    parser.add_argument("image_dir", help="Directory with images")
    
    args = parser.parse_args()
    
    main(args.input_path, args.output_path, args.csv_path, args.image_dir)

usage: ipykernel_launcher.py [-h] input_path output_path csv_path image_dir
ipykernel_launcher.py: error: the following arguments are required: input_path, output_path, csv_path, image_dir


SystemExit: 2

c:\Users\Gebruiker\AppData\Local\Programs\Python\Python311\Lib\site-packages\IPython\core\interactiveshell.py:3450: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
import argparse
from pathlib import Path
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

def load_data(csv_path, image_dir):
    df = pd.read_csv(csv_path)
    df['filepath'] = df['basename'].apply(lambda x: str(Path(image_dir) / x))
    return df

def create_data_generators(df, image_size=(128, 128), batch_size=32, validation_split=0.2):
    datagen = ImageDataGenerator(rescale=1./255, validation_split=validation_split)
    
    train_generator = datagen.flow_from_dataframe(
        df,
        x_col='filepath',
        y_col='deepest_name',
        target_size=image_size,
        batch_size=batch_size,
        class_mode='categorical',
        subset='training'
    )
    
    validation_generator = datagen.flow_from_dataframe(
        df,
        x_col='filepath',
        y_col='deepest_name',
        target_size=image_size,
        batch_size=batch_size,
        class_mode='categorical',
        subset='validation'
    )
    
    return train_generator, validation_generator

def build_model(input_shape, num_classes):
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(512, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])
    
    model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

def predict(model, inputs, image_size=(128, 128)):
    images = []
    for img_path in inputs:
        img = tf.keras.preprocessing.image.load_img(img_path, target_size=image_size)
        img = tf.keras.preprocessing.image.img_to_array(img)
        img = np.expand_dims(img, axis=0)
        img = img / 255.0
        images.append(img)
    
    images = np.vstack(images)
    predictions = model.predict(images)
    return predictions

def main(input_path, output_path, csv_path, image_dir):
    df = load_data(csv_path, image_dir)
    
    # Debugging line to ensure 'filepath' column exists
    print(df.head())
    
    train_generator, validation_generator = create_data_generators(df)
    
    input_shape = (128, 128, 3)
    num_classes = len(train_generator.class_indices)
    
    model = build_model(input_shape, num_classes)
    
    model.fit(train_generator, validation_data=validation_generator, epochs=10)
    
    inputs = list(Path(input_path).glob("*.jpg"))
    predictions = predict(model, inputs)
    
    values = {
        "image_uid": [Path(input).stem for input in inputs],
        "predictions": [list(train_generator.class_indices.keys())[pred] for pred in np.argmax(predictions, axis=1)]
    }
    
    df_predictions = pd.DataFrame(values)
    df_predictions.to_csv(output_path, index=False)

if __name__ == '__main__':
    parser = argparse.ArgumentParser()
    
    parser.add_argument("input_path", help="Path to input images")
    parser.add_argument("output_path", help="Path to output csv")
    parser.add_argument("csv_path", help="Path to CSV file with labels")
    parser.add_argument("image_dir", help="Directory with images")
    
    args = parser.parse_args()
    
    main(args.input_path, args.output_path, args.csv_path, args.image_dir)


In [ ]:
import argparse
from pathlib import Path
import pandas


def predict(inputs: Path):    
    inputs = list(inputs)
    
    # your algorithm goes here
    
    # delete and rewrite values with your predictions
    values = {
        "image_uid":[Path(input).stem for input in inputs],
        "level_0": ["Animalia"] * len(inputs),
        "level_0_probability": [1.0] * len(inputs)
    }

    return mapping_to_csv(values)

def mapping_to_csv(values: dict[str: any]):
    columns = ["image_uid","level_0","level_0_probability","level_1","level_1_probability","level_2","level_2_probability","level_3","level_3_probability","level_4","level_4_probability","level_5","level_5_probability"]

    return pandas.DataFrame(columns=columns, data=values)

if __name__ == '__main__':
    parser = argparse.ArgumentParser()

    parser.add_argument(
        "input_path", help="Path to input images"
    )

    parser.add_argument(
        "output_path", help="Path to output csv"
    )

    args = parser.parse_args()

    predict(Path(args.input_path).glob("*.jpg")).to_csv(args.output_path, index=False)